In [1]:
# Name: Ishan Vyas
# UCID: 30068270
# Course: CPSC 502.06
# Project: Regression testing for machine learning classifiers
# Supervisor: Dr. Frank Maurer

# Sample Program based around the 'diabetes' dataset.

In [2]:
# This is a simple diabetes predictor program that predicts 
# if a person has diabetes or not based on a KNN machine learning model.
# Database link: https://www.kaggle.com/datasets/mathchi/diabetes-data-set

In [3]:
# Import statements
from hashdf import HashableDataFrame

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

In [4]:
# This function loads the dataset
def load_dataset(fileName):
        return pd.read_csv(fileName)

In [5]:
# This function is responsible for finding the best K value for the given dataset
def find_bestK(start, end, df):
    data = df.loc[:, df.columns != 'Outcome']
    labels = df.loc[:, df.columns == 'Outcome']
    
    k_value_range = range(start,end)
    # Result scores
    k_value_scores = []
    K = 0
    for k in k_value_range:
        knn_model = KNeighborsClassifier(n_neighbors = k)
        accuracy = cross_val_score(knn_model, data, labels.values.ravel(), cv=10, scoring="accuracy")
        k_value_scores.append(accuracy.mean())

    K = k_value_scores.index(max(k_value_scores))+1
    return K

In [6]:
# This function is responsible for training a new classifier based on the provided dataset
def getModel(db, K):
    data = db.loc[:, db.columns != 'Outcome']
    labels = db.loc[:, db.columns == 'Outcome']
    
    X_train, X_test, y_train, y_test = train_test_split(data, labels.values.ravel(), test_size=0.25, random_state=1)

    neigh = KNeighborsClassifier(n_neighbors=K)  
    neigh.fit(X_train, y_train)

    Initial = neigh.predict(X_test)
    return neigh, Initial

In [7]:
# Load the diabetes dataset
df = load_dataset('diabetes.csv')
%store df

Stored 'df' (DataFrame)


In [8]:
# Show database information
df.info()
# Show the first 5 rows of the dataset
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [9]:
# Change the current dataframe into a hashable one so that we can pass it as a parameter 
df_hash = HashableDataFrame(df)
%store df_hash

Stored 'df_hash' (HashableDataFrame)


In [10]:
# Find the best K value
K = find_bestK(1,30, df_hash)
%store K

Stored 'K' (int)


In [11]:
# Get the classifer for the given dataset
diabetesPredictor = getModel(df_hash, K)

In [12]:
# Use the generated model to make predictions
# These are sample examples
# The idea here is that this model can be used as a component within a bigger project
# Thus we would like the model to behave in a predictable way
# If it doesn't, then the tester should be informed about the change in behavior
diabetesPredictor[0].predict([[0,80,40,35,0,43.1,0.58,20]])

array([0])

In [13]:
diabetesPredictor[0].predict([[1,100,50,20,100,43.1,0.688,30]])

array([0])

In [14]:
diabetesPredictor[0].predict([[2,120,60,20,160,33.1,0.288,40]])

array([0])

In [15]:
diabetesPredictor[0].predict([[0,140,90,15,168,26.1,2.888,50]])

array([1])

In [16]:
diabetesPredictor[0].predict([[4,160,40,0,90,23.1,1.288,60]])

array([1])

In [17]:
diabetesPredictor[0].predict([[5,180,50,35,80,28.1,3.288,50]])

array([0])